In [3]:
import numpy as np
import pandas as pd
from fuzzywuzzy import fuzz
from fuzzywuzzy import process
import swifter
from fastai.text import *
path = Path('/home/jupyter/tutorials/Thathasthu')
import gensim
from spellchecker import SpellChecker
spell = SpellChecker()

In [112]:
!pip install boto

In [4]:

train = pd.read_csv('./tathasthu/validation_set.csv')
manufacturers = pd.read_csv("./tathasthu/manufacturer_list.csv")
print(train.head())
print(manufacturers.head())

                                     article_name  \
0        BAGRRY MUESLI CHOCO DELIGHT BX 400g/500g   
1               HEN FRUIT LARGE WHITE EGG 30 PACK   
2  BIOTIQUE FACEWASH BIO PAPAYA EXFOLIATING 100ml   
3           TILDA BASMATI RICE KHUS LONG GRAIN 1K   
4             KARMIQ OLIVE OIL PURE REFINED BT 1L   

                            manufacturer_description  
0  Company Name - Bagrrys India Ltd<br />Membersh...  
1  Company Name - FOOD SERVICES<br />Membership D...  
2  Company Name - BIOVEDA ACTION RESEARCH PVT LTD...  
3  Company Name - TILDA HAIN INDIA PVT LTD<br />M...  
4  Company Name - Future Consumer Limited<br />Me...  
         brand_name
0     ORGANIC INDIA
1  head & shoulders
2          Sunfeast
3            Unibic
4           CREMICA


In [5]:
def clean_column(column, delimitter_list):
    column = column.str.lower()
    for delimitter in delimitter_list: 
        column = column.str.replace(delimitter,"")
    column = column.str.replace("  "," ")
    column = column.str.replace("  "," ")
    column = column.str.replace("  "," ")
    column = column.str.replace("  "," ")
    column = column.str.replace("  "," ")
    column = column.str.replace("  "," ")
    return column

def get_brand_name(article_name, choices):
    maxScore = 0
    maxChoice = ""
    for choice in choices:
        score = fuzz.token_set_ratio(" " + article_name + " ", " " + choice + " ")
        if score > maxScore or (score == maxScore and len(choice) > len(maxChoice)):
            maxScore = score
            maxChoice = choice
        if len(choice) > 5:
            concat_article_name = article_name.replace(" ", "")
            concat_choice = choice.replace(" ", "")
            score = 100 if concat_choice in concat_article_name else 0
            if score > maxScore or (score == maxScore and len(choice) > len(maxChoice)):
                maxScore = score
                maxChoice = choice
    return maxChoice, maxScore

def get_strict(article_name, choices):
    matches = []
    for choice in choices:
        choice1 = choice + " "
        choice2 = " " + choice + " "
        choice3 = " " + choice
        if article_name.startswith(choice1) or article_name.endswith(choice3) or choice2 in article_name:
            matches.append(choice)
    return matches

def remove_string(large, queries):
    for query in queries.split(" "):
        large = large.replace(query, "")
        large = large.replace(query+"s", "")
    large = large.replace("  ", " ")
    large = large.replace("  ", " ")
    large = large.replace("  ", " ")
    return large

def get_type(filtered_string):
    type_column = set()
    for word in filtered_string.split(' '):
        for seed in seed_choices:
            try: 
                similarity_score = model.similarity(seed, word)
            except KeyError:
                similarity_score = 0.0
            if similarity_score > 0.35:
                type_column.add(word)
    #         print(seed + " " + word + " " + str(similarity_score))
    s = list(type_column)
    string_array = ' '.join(str(e) for e in s)
    return string_array

def get_words_from_sentence(sentence):
    words = sentence.split(" ")
    returns = []
    misspelled = spell.unknown(words)
    for word in words:
        if len(word) > 2 and word not in misspelled and word not in stopwords:
            returns.append(word)
    return returns

seed_choices = ['oranges', 'apples', 'avocado', 'onion', 'tomato', 'pants', 'shirt', 'pink', 'blue']
seed_colors = ['red', 'blue', 'green']
seed_flavors = ['onion', 'tomato', 'apples', 'avocado']
seed_type = ['shirt', 'pant', 'soap', 'cosmetics']

def get_type_specific(filtered_string):
    type_column = set()
    for word in filtered_string.split(' '):
        for seed in seed_colors:
            try:
                similarity_score = model.similarity(seed, word)
            except KeyError:
                similarity_score = 0.0
            if similarity_score > 0.35:
                 type_column.add('color:' + word)
       
        for seed in seed_flavors:
            try:
                similarity_score = model.similarity(seed, word)
            except KeyError:
                similarity_score = 0.0
            if similarity_score > 0.35:
                type_column.add('flavor:' + word)

        for seed in seed_type:
            try:
                similarity_score = model.similarity(seed, word)
            except KeyError:
                similarity_score = 0.0
            if similarity_score > 0.35:
                type_column.add('type:' + word)
    
    s = list(type_column)
    string_array = ' '.join(str(e) for e in s)
    return string_array

In [6]:
train['manufacturer_description'] = clean_column(train['manufacturer_description'], [])

In [7]:
train["company_name"] = train.manufacturer_description.str.extract(r"company name - ([a-zA-Z-09\. \&]+)")

In [8]:
train["article_name"] = clean_column(train['article_name'], ["'", "(", ")", "/", "-"])

In [9]:
train['quantity'] = train['article_name'].str.extract(r'(\d.{0,4}(packs|m|ml|g|kg|l|s|p))')[0]

In [10]:
manufacturers["brand_name"] = clean_column(manufacturers['brand_name'], ["'", "(", ")", "/", "-"])

In [11]:
train["company_name"] = train["company_name"].fillna("")

In [12]:
train["article_name"] = train["article_name"].fillna("")

In [13]:
train['strict_transformed'] = train['article_name'].apply(lambda x: get_strict(x, manufacturers['brand_name']))

In [14]:
train['strict_company'] = train['company_name'].apply(lambda x: get_strict(x, manufacturers['brand_name']))

In [15]:
train = train[train["article_name"] != ""]

In [16]:
train2 = train

In [17]:
train['fuzzy_transformed'] = train['article_name'].swifter.apply(lambda x: get_brand_name(x, manufacturers['brand_name']))

Pandas Apply: 100%|██████████| 35/35 [00:00<00:00, 65.15it/s]


In [18]:
train['fuzzy_company'] = train['company_name'].swifter.apply(lambda x: get_brand_name(x, manufacturers['brand_name']))

Pandas Apply: 100%|██████████| 35/35 [00:00<00:00, 76.33it/s]


In [19]:
train = train.filter(['article_name','company_name', 'quantity', 'strict_transformed', 'strict_company', 'fuzzy_transformed', 'fuzzy_company']).join(train.fuzzy_transformed.apply(lambda loc: pd.Series(loc, index=['fuzzy_brand', 'brand_score']))).join(train.fuzzy_company.apply(lambda loc: pd.Series(loc, index=['f_company', 'company_score'])))



In [20]:
del train['fuzzy_transformed']
del train['fuzzy_company']

In [21]:
def get_brand_from_map(article_name):
    brand_map = {
        "ph": "premium harvest",
        "ttrt": "tasty treat",
        "gh": "golden harvest",
        "on1y": "only"
    }
    for key in brand_map:
        if key in article_name:
            return brand_map[key]
    return None

In [22]:
train3 = train

In [23]:
train

,article_name,company_name,quantity,strict_transformed,strict_company,fuzzy_brand,brand_score,f_company,company_score
0,bagrry muesli choco delight bx 400g500g,bagrrys india ltd,400g,[],[bagrrys],axe signature gold,35,bagrrys,100
1,hen fruit large white egg 30 pack,food services,30 p,"[hen fruit, fruit]",[],hen fruit,100,old spice,64
2,biotique facewash bio papaya exfoliating 100ml,bioveda action research pvt ltd,100ml,[biotique],[],biotique,100,oleev active,47
3,tilda basmati rice khus long grain 1k,tilda hain india pvt ltd,NaN,[tilda],[tilda],tilda,100,tilda,100
4,karmiq olive oil pure refined bt 1l,future consumer limited,1l,[karmiq],[],karmiq,100,center fruit,51
5,vlcc body ltn cocoa butter spf15 350ml,vlcc personal care ltd,5 350m,[vlcc],[vlcc],vlcc,100,vlcc,100
6,american garden cheese microwave popcorn 297g,bajoria foods pvt ltd,297g,"[american garden, garden]",[],american garden,100,hr foods,77
7,garnier fc scrub pure active apricot 50g,loreal india pvt ltd,50g,[garnier],[],garnier,100,loreal paris,67
8,quaker oats kesar kishmish 40g,fritolays,40g,[quaker],[],quaker,100,fruittella,63
9,nissin cup noodles tangy tomato cp 70g,indo nissin,70g,[nissin],[nissin],nissin,100,nissin,100


In [24]:
train["final_brand"] = train.swifter.apply(lambda x: x.fuzzy_brand if x.brand_score > 50 else x.strict_company[0] if len(x.strict_company) > 0 else get_brand_from_map(x.article_name), axis=1)


Pandas Apply: 100%|██████████| 35/35 [00:00<00:00, 13488.99it/s]


In [25]:
train = train.filter(["article_name", "quantity", "final_brand"])

In [26]:
train["filtered"] = train.apply(lambda x: x.article_name if x.final_brand is None else remove_string(x.article_name, x.final_brand), axis=1)


In [27]:
train["filtered"] = train.apply(lambda x: remove_string(x.filtered, str(x.quantity)), axis=1)

In [28]:
train.to_csv("train_quantity.csv", index=False)

In [29]:
train = pd.read_csv("train_quantity.csv")

In [30]:
learn = load_learner(path)

In [31]:
train["final_brand"] = train.swifter.apply(lambda x: "" if x.final_brand == None else x.final_brand, axis=1)

Pandas Apply: 100%|██████████| 35/35 [00:00<00:00, 17865.48it/s]


In [32]:
train['final_brand'] = train['final_brand'].fillna('')

In [33]:
train['filtered'] = train['filtered'].fillna('')

In [34]:
train['function_dl'] = train.swifter.apply(lambda x: str(learn.predict(x['final_brand']+' '+x['filtered'])[0]), axis=1)

Pandas Apply: 100%|██████████| 35/35 [00:01<00:00, 19.22it/s]


In [35]:
function_df = pd.read_csv("./classes")

In [36]:
function_filtered = pd.merge(train,function_df, how='left', left_on = 'function_dl', right_on = 'class1')

In [37]:
function_filtered['class1'] = clean_column(function_filtered['class1'], ["'", "(", ")", "/", "&", ","])

In [38]:
function_filtered['class2'] = clean_column(function_filtered['class2'], ["'", "(", ")", "/", "&", ","])

In [39]:
function_filtered['appended_class'] = function_filtered.apply(lambda x: x.class1 + " " + x.class2, axis=1)

In [40]:
function_filtered["cleaned"] = function_filtered.apply(lambda x: remove_string(x.filtered, x.appended_class), axis=1)

In [41]:
final1_data = function_filtered.filter(['article_name', 'cleaned', 'quantity', 'final_brand', 'class1', 'class2'])

In [42]:
final1_data.to_csv('./final1_data.csv', index=False)

In [43]:
final1_data = pd.read_csv('./final1_data.csv')

In [44]:
model = gensim.models.KeyedVectors.load_word2vec_format('./google.bin', binary=True)  



In [45]:
seed_choices = ['oranges', 'apples', 'avocado', 'onion', 'tomato', 'pants', 'shirt', 'pink', 'blue']
get_type("kitkat dessert delight truffle")

'truffle dessert'

In [46]:
final1_data["cleaned"] = final1_data.cleaned.fillna("")

In [47]:
final1_data['type'] = final1_data.swifter.apply(lambda x:get_type_specific(x.cleaned), axis=1)

Pandas Apply: 100%|██████████| 35/35 [00:00<00:00, 876.80it/s]


In [48]:
final1_data

,article_name,cleaned,quantity,final_brand,class1,class2,type
0,bagrry muesli choco delight bx 400g500g,bagrry muesli choco delight bx 500g,400g,bagrrys,breakfast cereals,branded food,flavor:choco flavor:muesli
1,hen fruit large white egg 30 pack,large white egg ack,30 p,hen fruit,chocolates cakes confectioneries,branded food,color:white flavor:egg
2,biotique facewash bio papaya exfoliating 100ml,facewash bio papaya exfoliating,100ml,biotique,baby care,baby care,type:exfoliating flavor:papaya
3,tilda basmati rice khus long grain 1k,basmati rice khus long grain 1k,NaN,tilda,organic grains,grocery,flavor:khus flavor:rice type:khus flavor:grain...
4,karmiq olive oil pure refined bt 1l,olive pure refined bt,1l,karmiq,edible oil ghee,grocery,color:olive flavor:olive
5,vlcc body ltn cocoa butter spf15 350ml,body ltn cocoa butter spf1 30ml,5 350m,vlcc,skin care,beauty personal care,flavor:butter
6,american garden cheese microwave popcorn 297g,cheese microwave popcorn,297g,american garden,jams spreads honey,branded food,flavor:cheese
7,garnier fc scrub pure active apricot 50g,fc scrub pure active apricot,50g,garnier,skin care,beauty personal care,flavor:apricot
8,quaker oats kesar kishmish 40g,oats kesar kishmish,40g,quaker,breakfast cereals,branded food,flavor:kesar flavor:oats flavor:kishmish
9,nissin cup noodles tangy tomato cp 70g,cup tangy tomato cp,70g,nissin,noodles pastas vermicelli,branded food,flavor:tomato flavor:tangy


In [49]:
final1_data['cleaned_more'] = final1_data.swifter.apply(lambda x: remove_string(x.cleaned, x.type), axis=1)

Pandas Apply: 100%|██████████| 35/35 [00:00<00:00, 11154.22it/s]


In [50]:
final1_data

,article_name,cleaned,quantity,final_brand,class1,class2,type,cleaned_more
0,bagrry muesli choco delight bx 400g500g,bagrry muesli choco delight bx 500g,400g,bagrrys,breakfast cereals,branded food,flavor:choco flavor:muesli,bagrry muesli choco delight bx 500g
1,hen fruit large white egg 30 pack,large white egg ack,30 p,hen fruit,chocolates cakes confectioneries,branded food,color:white flavor:egg,large white egg ack
2,biotique facewash bio papaya exfoliating 100ml,facewash bio papaya exfoliating,100ml,biotique,baby care,baby care,type:exfoliating flavor:papaya,facewash bio papaya exfoliating
3,tilda basmati rice khus long grain 1k,basmati rice khus long grain 1k,NaN,tilda,organic grains,grocery,flavor:khus flavor:rice type:khus flavor:grain...,basmati rice khus long grain 1k
4,karmiq olive oil pure refined bt 1l,olive pure refined bt,1l,karmiq,edible oil ghee,grocery,color:olive flavor:olive,olive pure refined bt
5,vlcc body ltn cocoa butter spf15 350ml,body ltn cocoa butter spf1 30ml,5 350m,vlcc,skin care,beauty personal care,flavor:butter,body ltn cocoa butter spf1 30ml
6,american garden cheese microwave popcorn 297g,cheese microwave popcorn,297g,american garden,jams spreads honey,branded food,flavor:cheese,cheese microwave popcorn
7,garnier fc scrub pure active apricot 50g,fc scrub pure active apricot,50g,garnier,skin care,beauty personal care,flavor:apricot,fc scrub pure active apricot
8,quaker oats kesar kishmish 40g,oats kesar kishmish,40g,quaker,breakfast cereals,branded food,flavor:kesar flavor:oats flavor:kishmish,oats kesar kishmish
9,nissin cup noodles tangy tomato cp 70g,cup tangy tomato cp,70g,nissin,noodles pastas vermicelli,branded food,flavor:tomato flavor:tangy,cup tangy tomato cp


In [51]:
stopwords = ["i", "me", "my", "myself", "we", "our", "ours", "ourselves", "you", "your", "yours", "yourself", "yourselves", "he", "him", "his", "himself", "she", "her", "hers", "herself", "it", "its", "itself", "they", "them", "their", "theirs", "themselves", "what", "which", "who", "whom", "this", "that", "these", "those", "am", "is", "are", "was", "were", "be", "been", "being", "have", "has", "had", "having", "do", "does", "did", "doing", "a", "an", "the", "and", "but", "if", "or", "because", "as", "until", "while", "of", "at", "by", "for", "with", "about", "against", "between", "into", "through", "during", "before", "after", "above", "below", "to", "from", "up", "down", "in", "out", "on", "off", "over", "under", "again", "further", "then", "once", "here", "there", "when", "where", "why", "how", "all", "any", "both", "each", "few", "more", "most", "other", "some", "such", "no", "nor", "not", "only", "own", "same", "so", "than", "too", "very", "s", "t", "can", "will", "just", "don", "should", "now"]

In [52]:
final1_data["get_cleaned"] = final1_data.swifter.apply(lambda x: get_words_from_sentence(x.cleaned_more), axis=1)

Pandas Apply: 100%|██████████| 35/35 [00:00<00:00, 12449.17it/s]


In [53]:
final1_data.to_csv("./final_datav1.1.2demo.csv", index=False)

In [54]:
final1_data.head(100)

,article_name,cleaned,quantity,final_brand,class1,class2,type,cleaned_more,get_cleaned
0,bagrry muesli choco delight bx 400g500g,bagrry muesli choco delight bx 500g,400g,bagrrys,breakfast cereals,branded food,flavor:choco flavor:muesli,bagrry muesli choco delight bx 500g,"[muesli, delight]"
1,hen fruit large white egg 30 pack,large white egg ack,30 p,hen fruit,chocolates cakes confectioneries,branded food,color:white flavor:egg,large white egg ack,"[large, white, egg, ack]"
2,biotique facewash bio papaya exfoliating 100ml,facewash bio papaya exfoliating,100ml,biotique,baby care,baby care,type:exfoliating flavor:papaya,facewash bio papaya exfoliating,"[bio, papaya]"
3,tilda basmati rice khus long grain 1k,basmati rice khus long grain 1k,NaN,tilda,organic grains,grocery,flavor:khus flavor:rice type:khus flavor:grain...,basmati rice khus long grain 1k,"[basmati, rice, long, grain]"
4,karmiq olive oil pure refined bt 1l,olive pure refined bt,1l,karmiq,edible oil ghee,grocery,color:olive flavor:olive,olive pure refined bt,"[olive, pure, refined]"
5,vlcc body ltn cocoa butter spf15 350ml,body ltn cocoa butter spf1 30ml,5 350m,vlcc,skin care,beauty personal care,flavor:butter,body ltn cocoa butter spf1 30ml,"[body, cocoa, butter]"
6,american garden cheese microwave popcorn 297g,cheese microwave popcorn,297g,american garden,jams spreads honey,branded food,flavor:cheese,cheese microwave popcorn,"[cheese, microwave, popcorn]"
7,garnier fc scrub pure active apricot 50g,fc scrub pure active apricot,50g,garnier,skin care,beauty personal care,flavor:apricot,fc scrub pure active apricot,"[scrub, pure, active, apricot]"
8,quaker oats kesar kishmish 40g,oats kesar kishmish,40g,quaker,breakfast cereals,branded food,flavor:kesar flavor:oats flavor:kishmish,oats kesar kishmish,[oats]
9,nissin cup noodles tangy tomato cp 70g,cup tangy tomato cp,70g,nissin,noodles pastas vermicelli,branded food,flavor:tomato flavor:tangy,cup tangy tomato cp,"[cup, tangy, tomato]"


In [55]:
output = final1_data.filter(['article_name', 'quantity', 'final_brand', 'class1', 'class2', 'type', 'get_cleaned'])

In [56]:
output = output.rename(columns={"final_brand": "brand", "class1": "function", "class2": "function_parent", "type": "type-flavour-colour", "get_cleaned": "misc"})

In [57]:
output["misc"] = output.misc.swifter.apply(lambda x: ",".join(x))

Pandas Apply: 100%|██████████| 35/35 [00:00<00:00, 49228.92it/s]


In [58]:
output.function = output.swifter.apply(lambda x: x.function + " | " + x.function_parent, axis=1)

In [59]:
del output["function_parent"]

In [60]:
output.to_csv("./output_demo.csv", index=False)

In [61]:
output

,article_name,quantity,brand,function,type-flavour-colour,misc
0,bagrry muesli choco delight bx 400g500g,400g,bagrrys,breakfast cereals | branded food,flavor:choco flavor:muesli,"muesli,delight"
1,hen fruit large white egg 30 pack,30 p,hen fruit,chocolates cakes confectioneries | branded food,color:white flavor:egg,"large,white,egg,ack"
2,biotique facewash bio papaya exfoliating 100ml,100ml,biotique,baby care | baby care,type:exfoliating flavor:papaya,"bio,papaya"
3,tilda basmati rice khus long grain 1k,NaN,tilda,organic grains | grocery,flavor:khus flavor:rice type:khus flavor:grain...,"basmati,rice,long,grain"
4,karmiq olive oil pure refined bt 1l,1l,karmiq,edible oil ghee | grocery,color:olive flavor:olive,"olive,pure,refined"
5,vlcc body ltn cocoa butter spf15 350ml,5 350m,vlcc,skin care | beauty personal care,flavor:butter,"body,cocoa,butter"
6,american garden cheese microwave popcorn 297g,297g,american garden,jams spreads honey | branded food,flavor:cheese,"cheese,microwave,popcorn"
7,garnier fc scrub pure active apricot 50g,50g,garnier,skin care | beauty personal care,flavor:apricot,"scrub,pure,active,apricot"
8,quaker oats kesar kishmish 40g,40g,quaker,breakfast cereals | branded food,flavor:kesar flavor:oats flavor:kishmish,oats
9,nissin cup noodles tangy tomato cp 70g,70g,nissin,noodles pastas vermicelli | branded food,flavor:tomato flavor:tangy,"cup,tangy,tomato"
